In [1]:
from pathlib import Path
import sys
import os
root_dir = str(Path(os.getcwd()).parent)
sys.path.append(root_dir)

from report.load.load_data import load_yysj_data, get_trade_date_lag, calc_tech_factor
from report.util import get_trade_date

import multiprocessing as mp
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from functools import partial
import datetime 

import warnings
warnings.filterwarnings('ignore')


In [2]:
# --------------------------------读取预约数据并存储为yysj.pq--------------------------------
yysj_file = f'{root_dir}/data/intermediate_results/yysj.parquet'
if os.path.isfile(yysj_file):
	yysj = pq.read_table(yysj_file).to_pandas()
else:
	with mp.Pool() as pool:
		yysj = pool.map(load_yysj_data, range(2007,2025))
	yysj = pd.concat(yysj).reset_index(drop=True)
	yysj.to_parquet(yysj_file)
yysj

,stock_symbol,首次预约时间,一次变更日期,二次变更日期,三次变更日期,实际披露时间,report_type,start_day,deadline
0,000001,2007-03-22,None,None,None,2007-03-22,20061231,2007-01-01,2007-04-30
1,000002,2007-03-20,None,None,None,2007-03-20,20061231,2007-01-01,2007-04-30
2,000004,2007-04-27,None,None,None,2007-04-27,20061231,2007-01-01,2007-04-30
3,000006,2007-03-24,None,None,None,2007-03-24,20061231,2007-01-01,2007-04-30
4,000007,2007-04-20,None,None,None,2007-04-20,20061231,2007-01-01,2007-04-30
...,...,...,...,...,...,...,...,...,...
209554,688799,2024-10-29,None,None,None,2024-10-29,20240930,2024-10-01,2024-10-31
209555,688800,2024-10-25,None,None,None,2024-10-25,20240930,2024-10-01,2024-10-31
209556,688819,2024-10-30,None,None,None,2024-10-30,20240930,2024-10-01,2024-10-31
209557,688981,2024-11-08,None,None,None,2024-11-08,20240930,2024-10-01,2024-10-31


In [3]:
# --------------------------------生成预约时间衍生指标并存储为yysj_pro.pq--------------------------------
yysj_pro_file = f'{root_dir}/data/intermediate_results/yysj_pro.parquet'
if os.path.isfile(yysj_pro_file):
	yysj_pro = pq.read_table(yysj_pro_file).to_pandas()
else:
	yysj_pro = yysj.dropna(subset=['stock_symbol', '首次预约时间', '实际披露时间']).reset_index(drop=True).copy()
	# 生成最终预约时间
	yysj_pro['latest_book_date'] = np.nan
	yysj_pro['latest_book_date'] = yysj_pro['latest_book_date'].fillna(yysj_pro['三次变更日期'])
	yysj_pro['latest_book_date'] = yysj_pro['latest_book_date'].fillna(yysj_pro['二次变更日期'])
	yysj_pro['latest_book_date'] = yysj_pro['latest_book_date'].fillna(yysj_pro['一次变更日期'])
	yysj_pro['latest_book_date'] = yysj_pro['latest_book_date'].fillna(yysj_pro['首次预约时间'])
	# 预约时间排序
	yysj_pro['book_time_rank_first'] = yysj_pro.groupby('report_type')['首次预约时间'].rank(ascending=True, pct=True, method='min')
	yysj_pro['book_time_rank_last'] = yysj_pro.groupby('report_type')['latest_book_date'].rank(ascending=True, pct=True, method='min')
	yysj_pro['report_time_rank_last'] = yysj_pro.groupby('report_type')['实际披露时间'].rank(ascending=True, pct=True, method='min')
	yysj_pro['days_to_deadline'] = (yysj_pro['deadline'] - yysj_pro['首次预约时间']).apply(lambda x: x.days)
	yysj_pro['type'] = yysj_pro['report_type'].str[-4:]
	yysj_pro['days_to_startline'] = (yysj_pro['首次预约时间'] - yysj_pro['start_day']).apply(lambda x: x.days)
	yysj_pro = yysj_pro[['stock_symbol', 'report_type', 'type', 'book_time_rank_first', 'book_time_rank_last', 'report_time_rank_last', 'days_to_startline', 'days_to_deadline']]
	yysj_pro.to_parquet(yysj_pro_file)
yysj_pro

,stock_symbol,report_type,type,book_time_rank_first,book_time_rank_last,report_time_rank_last,days_to_startline,days_to_deadline
0,000001,20061231,1231,0.280000,0.280000,0.280784,80,39
1,000002,20061231,1231,0.247843,0.247843,0.247843,78,41
2,000004,20061231,1231,0.873725,0.873725,0.874510,116,3
3,000006,20061231,1231,0.309804,0.309804,0.309804,82,37
4,000007,20061231,1231,0.734118,0.734118,0.737255,109,10
...,...,...,...,...,...,...,...,...
209547,688799,20240930,0930,0.355142,0.350833,0.350833,28,2
209548,688800,20240930,0930,0.100686,0.100881,0.100881,24,6
209549,688819,20240930,0930,0.529481,0.526543,0.526543,29,1
209550,688981,20240930,0930,0.999412,0.999412,0.999412,38,-8


In [4]:
# --------------------------------生成不同滞后期的对应时间并存储为trade_date_lag.pq--------------------------------
trade_date_lag_file = f'{root_dir}/data/intermediate_results/trade_date_lag.parquet'
if os.path.isfile(trade_date_lag_file):
	trade_date_lag = pq.read_table(trade_date_lag_file).to_pandas()
else:
	with mp.Pool() as pool:
		partial_func = partial(get_trade_date_lag, yysj)
		trade_date_lag = pool.map(partial_func, range(-30,31))
		trade_date_lag = pd.concat(trade_date_lag).reset_index(drop=True)
	trade_date_lag.to_parquet(trade_date_lag_file)
trade_date_lag

,stock_symbol,report_type,trade_date,lag
0,000001,20061231,2007-02-01,-30
1,000002,20061231,2007-01-30,-30
2,000004,20061231,2007-03-16,-30
3,000006,20061231,2007-02-05,-30
4,000007,20061231,2007-03-09,-30
...,...,...,...,...
12782728,688799,20240930,2024-12-10,30
12782729,688800,20240930,2024-12-06,30
12782730,688819,20240930,2024-12-11,30
12782731,688981,20240930,2024-12-20,30


In [4]:
# --------------------------------读取个股回报率数据并存储为retrun_data.pq--------------------------------
# Dretwd [考虑现金红利再投资的日个股回报率] - 上市首日的前收盘价取招股价,字段说明见说明书“日个股回报率的计算”。
# Dretnd [不考虑现金红利的日个股回报率] - 上市首日的前收盘价取招股价,字段说明见说明书“日个股回报率的计算”。
# Markettype [市场类型] - 1=上证A股市场 (不包含科创板），2=上证B股市场，4=深证A股市场（不包含创业板），8=深证B股市场，16=创业板， 32=科创板，64=北证A股市场。
# Trdsta [交易状态] - 1=正常交易，2=ST，3＝*ST，4＝S（2006年10月9日及之后股改未完成），5＝SST，6＝S*ST，7=G（2006年10月9日之前已完成股改），8=GST，9=G*ST，10=U（2006年10月9日之前股改未完成），11=UST，12=U*ST，13=N，14=NST，15=N*ST，16=PT
return_data_file = f'{root_dir}/data/intermediate_results/return_data.parquet'
if os.path.isfile(return_data_file):
	return_data = pq.read_table(return_data_file).to_pandas()
else:
	return_data_file_list = [f'{root_dir}/data/raw_data/return_data/TRD_Dalyr01.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr02.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr03.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr04.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr05.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr06.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr07.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr08.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr09.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr10.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr11.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr12.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr13.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr14.xlsx',
							f'{root_dir}/data/raw_data/return_data/TRD_Dalyr15.xlsx',
							]
	return_data_list = []
	for return_data_1 in return_data_file_list:
		return_data_1 = pd.read_excel(return_data_1)
		return_data_1 = return_data_1.iloc[2:]
		return_data_1['stock_symbol'] = return_data_1['Stkcd'].str[:6]
		return_data_1['trade_date'] = pd.to_datetime(return_data_1['Trddt']).dt.date
		return_data_1['rets1'] = return_data_1['Dretwd'].astype(float)
		return_data_1['rets2'] = return_data_1['Dretnd'].astype(float)
		return_data_1['state'] = return_data_1['Trdsta'].astype(int)
		return_data_1['close'] = return_data_1['Clsprc'].astype(float)
		return_data_1['pre_close'] = return_data_1['PreClosePrice'].astype(float)
		return_data_1['volume'] = return_data_1['Dnshrtrd'].astype(float)
		return_data_1['amount'] = return_data_1['Dnvaltrd'].astype(float)
		return_data_1['float_market_cap'] = return_data_1['Dsmvosd'].astype(float)
		return_data_1['market_cap'] = return_data_1['Dsmvtll'].astype(float)

		return_data_1 = return_data_1.loc[return_data_1['state'] == 1, ['stock_symbol', 'trade_date', 'state', 'rets1', 'rets2', 'close', 'pre_close', 'volume', 'amount', 'float_market_cap', 'market_cap']]
		return_data_list.append(return_data_1)
	return_data = pd.concat(return_data_list)
	return_data.drop_duplicates()
	return_data.to_parquet(return_data_file)
return_data

,stock_symbol,trade_date,state,rets1,rets2,close,pre_close,volume,amount,float_market_cap,market_cap
2,000001,2004-12-17,1,-0.011445,-0.011445,6.91,6.99,2059533.0,1.430900e+07,9738691.18,13445631.05
3,000001,2004-12-20,1,-0.007236,-0.007236,6.86,6.91,3002280.0,2.064922e+07,9668223.08,13348339.94
4,000001,2004-12-21,1,0.000000,0.000000,6.86,6.86,2420394.0,1.656034e+07,9668223.08,13348339.94
5,000001,2004-12-22,1,0.016035,0.016035,6.97,6.86,3833753.0,2.642449e+07,9823252.90,13562380.38
6,000001,2004-12-23,1,-0.012912,-0.012912,6.88,6.97,1738410.0,1.205999e+07,9696410.32,13387256.39
...,...,...,...,...,...,...,...,...,...,...,...
834462,920128,2024-12-10,1,-0.003368,-0.003368,35.51,35.63,3141431.0,1.138672e+08,607221.00,2787179.90
834463,920128,2024-12-11,1,-0.038862,-0.038862,34.13,35.51,2577686.0,8.886673e+07,583623.00,2678863.70
834464,920128,2024-12-12,1,0.020217,0.020217,34.82,34.13,2296236.0,8.011653e+07,595422.00,2733021.80
834465,920128,2024-12-13,1,-0.007754,-0.007754,34.55,34.82,1619491.0,5.651567e+07,590805.00,2711829.50


In [9]:
# --------------------------------处理股票信息数据并存储为stk_info.pq--------------------------------
stk_info_file = f'{root_dir}/data/intermediate_results/stk_info.parquet'
if os.path.isfile(stk_info_file):
	stk_info = pq.read_table(stk_info_file).to_pandas()
else:
	stk_info = pd.read_excel(f'{root_dir}/data/raw_data/stk_info.xlsx').iloc[2:]
	stk_info['stock_symbol'] = stk_info['Stkcd']
	stk_info['listed_date'] = pd.to_datetime(stk_info['Listdt']).dt.date
	stk_info = stk_info[['stock_symbol', 'listed_date', 'Indcd', 'Statco', 'Markettype']]
	stk_info.to_parquet(stk_info_file)
stk_info

,stock_symbol,listed_date,Indcd,Statco,Markettype
2,000001,1991-04-03,0001,A,4
3,000002,1991-01-29,0003,A,4
4,000003,1991-07-03,0004,D,4
5,000004,1991-01-14,0002,A,4
6,000005,1990-12-10,0002,D,4
...,...,...,...,...,...
5782,920099,2024-09-25,0002,A,64
5783,920106,2024-12-18,0005,NaN,64
5784,920111,2024-11-11,0005,A,64
5785,920118,2024-08-22,0005,A,64


In [3]:
# --------------------------------处理财报因子数据并存储为factor_data_fundamentall.pq--------------------------------
factor_data_fundamentall_file = f'{root_dir}/data/intermediate_results/factor_data_fundamentall.parquet'
if os.path.isfile(factor_data_fundamentall_file):
	factor_data_fundamentall = pq.read_table(factor_data_fundamentall_file).to_pandas()
else:
	balance = pd.read_excel(f'{root_dir}/data/raw_data/report_data/FS_Combas.xlsx').iloc[2:]
	balance['stock_symbol'] = balance['Stkcd']
	balance['report_type'] = balance['Accper']
	balance['asset'] = balance['A001000000'].astype(float)
	balance['equity'] = balance['A003000000'].astype(float)
	balance = balance[['stock_symbol', 'report_type', 'asset', 'equity']]
	balance = balance.drop_duplicates(subset=['stock_symbol', 'report_type'], keep='last')

	income = pd.read_excel(f'{root_dir}/data/raw_data/report_data/FS_Comins.xlsx').iloc[2:]
	income['stock_symbol'] = income['Stkcd']
	income['report_type'] = income['Accper']
	income['net_profit'] = income['B002000000'].astype(float)
	income['basic_earnings_per_share'] = income['B003000000'].astype(float)
	income = income[['stock_symbol', 'report_type', 'net_profit', 'basic_earnings_per_share']]
	income = income.drop_duplicates(subset=['stock_symbol', 'report_type'], keep='last')

	factor_data_fundamentall = pd.merge(left=balance, right=income, on=['stock_symbol', 'report_type']).sort_values(by=['stock_symbol', 'report_type'])
	factor_data_fundamentall['report_type'] = factor_data_fundamentall['report_type'].str.replace('-','')
	factor_data_fundamentall['size'] = np.log(factor_data_fundamentall['asset'])
	factor_data_fundamentall['roe'] = factor_data_fundamentall['net_profit'] / factor_data_fundamentall['equity']
	factor_data_fundamentall['ue'] = factor_data_fundamentall['basic_earnings_per_share'] / factor_data_fundamentall['basic_earnings_per_share'].shift(4) - 1
	factor_data_fundamentall['ia'] = factor_data_fundamentall['asset'] / factor_data_fundamentall['asset'].shift(4) - 1
	factor_data_fundamentall['report_type_lag1'] = factor_data_fundamentall['report_type'].astype(int)
	factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('0331'), 'report_type_lag1'] = factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('0331'), 'report_type_lag1'] - 9100
	factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('0630'), 'report_type_lag1'] = factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('0630'), 'report_type_lag1'] - 299
	factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('0930'), 'report_type_lag1'] = factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('0930'), 'report_type_lag1'] - 300
	factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('1231'), 'report_type_lag1'] = factor_data_fundamentall.loc[factor_data_fundamentall['report_type'].str.endswith('1231'), 'report_type_lag1'] - 301
	factor_data_fundamentall['report_type_lag1'] = factor_data_fundamentall['report_type_lag1'].astype(str)
	factor_data_fundamentall.to_parquet(factor_data_fundamentall_file)

factor_data_fundamentall

,stock_symbol,report_type,asset,equity,net_profit,basic_earnings_per_share,size,roe,ue,ia,report_type_lag1
0,000001,19911231,4.354460e+09,5.779600e+08,NaN,NaN,22.194466,NaN,NaN,NaN,19910930
1,000001,19921231,7.522847e+09,5.456622e+08,NaN,NaN,22.741211,NaN,NaN,NaN,19920930
2,000001,19931231,9.337871e+09,1.189130e+09,2.733111e+08,NaN,22.957344,0.229841,NaN,NaN,19930930
3,000001,19940630,1.246595e+10,1.512913e+09,NaN,NaN,23.246267,NaN,NaN,NaN,19940331
4,000001,19941231,1.548841e+10,1.659833e+09,3.563280e+08,NaN,23.463358,0.214677,NaN,2.556908,19940930
...,...,...,...,...,...,...,...,...,...,...,...
338843,920019,20240930,1.259091e+09,7.423134e+08,3.164510e+07,NaN,20.953656,0.042630,NaN,0.767093,20240630
338844,920088,20240930,5.812421e+08,5.056478e+08,3.181170e+07,NaN,20.180678,0.062913,NaN,0.489580,20240630
338845,920099,20240930,8.020667e+08,7.580631e+08,6.518883e+07,NaN,20.502702,0.085994,NaN,0.765956,20240630
338846,920118,20240630,1.118436e+09,4.555119e+08,3.805434e+07,NaN,20.835197,0.083542,NaN,1.415609,20240331


In [2]:
# --------------------------------处理技术因子数据并存储为factor_data_tech.pq--------------------------------
factor_data_tech_file = f'{root_dir}/data/intermediate_results/factor_data_tech.parquet'
return_data_file = f'{root_dir}/data/intermediate_results/return_data.parquet'
if os.path.isfile(factor_data_tech_file):
	factor_data_tech = pq.read_table(factor_data_tech_file).to_pandas()
else:
	factor_data_tech = pq.read_table(return_data_file).to_pandas()
	factor_data_tech = factor_data_tech.groupby('stock_symbol').apply(calc_tech_factor)
	factor_data_tech = factor_data_tech.reset_index(drop=True)
	factor_data_tech.to_parquet(factor_data_tech_file)
factor_data_tech

,stock_symbol,trade_date,state,rets1,rets2,close,pre_close,volume,amount,float_market_cap,market_cap,turnonver,vol5,volt20,mtm,turnonver2_6,turnonver12_61,abnormal_volume
0,000001,2004-12-17,1,-0.011445,-0.011445,6.91,6.99,2059533.0,1.430900e+07,9738691.18,13445631.05,0.211479,NaN,NaN,NaN,NaN,NaN,NaN
1,000001,2004-12-20,1,-0.007236,-0.007236,6.86,6.91,3002280.0,2.064922e+07,9668223.08,13348339.94,0.310531,NaN,NaN,NaN,NaN,NaN,NaN
2,000001,2004-12-21,1,0.000000,0.000000,6.86,6.86,2420394.0,1.656034e+07,9668223.08,13348339.94,0.250345,NaN,NaN,NaN,NaN,NaN,NaN
3,000001,2004-12-22,1,0.016035,0.016035,6.97,6.86,3833753.0,2.642449e+07,9823252.90,13562380.38,0.390273,NaN,NaN,NaN,NaN,NaN,NaN
4,000001,2004-12-23,1,-0.012912,-0.012912,6.88,6.97,1738410.0,1.205999e+07,9696410.32,13387256.39,0.179284,0.268383,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13365421,920128,2024-12-10,1,-0.003368,-0.003368,35.51,35.63,3141431.0,1.138672e+08,607221.00,2787179.90,5.173456,7.910829,NaN,NaN,8.868222,NaN,NaN
13365422,920128,2024-12-11,1,-0.038862,-0.038862,34.13,35.51,2577686.0,8.886673e+07,583623.00,2678863.70,4.416697,7.239630,NaN,NaN,7.910829,NaN,NaN
13365423,920128,2024-12-12,1,0.020217,0.020217,34.82,34.13,2296236.0,8.011653e+07,595422.00,2733021.80,3.856485,6.513914,NaN,NaN,7.239630,NaN,NaN
13365424,920128,2024-12-13,1,-0.007754,-0.007754,34.55,34.82,1619491.0,5.651567e+07,590805.00,2711829.50,2.741160,4.988754,NaN,-0.251192,6.513914,NaN,NaN


In [2]:
# --------------------------------处理财报数据并存储为report_data.pq--------------------------------
report_data_file = f'{root_dir}/data/intermediate_results/report_data.parquet'
if os.path.isfile(report_data_file):
	report_data = pq.read_table(report_data_file).to_pandas()
else:
	report_data = pd.read_csv(f'{root_dir}/data/raw_data/report_data.csv')
	report_data['stock_symbol'] = report_data['order_book_id'].str[:6]
	report_data['report_type'] = report_data['quarter'].str.replace(r'(\d{4})q1', r'\g<1>0331', regex=True)
	report_data['report_type'] = report_data['report_type'].str.replace(r'(\d{4})q2', r'\g<1>0630', regex=True)
	report_data['report_type'] = report_data['report_type'].str.replace(r'(\d{4})q3', r'\g<1>0930', regex=True)
	report_data['report_type'] = report_data['report_type'].str.replace(r'(\d{4})q4', r'\g<1>1231', regex=True)
	report_data.sort_values(by=['stock_symbol', 'report_type'], inplace=True)
	report_data = report_data[['stock_symbol', 'report_type', 'total_liabilities', 'net_profit', 'total_assets', 'total_equity', 'basic_earnings_per_share']]
	report_data['roa'] = report_data['net_profit'] / report_data['total_assets']
	report_data['roe'] = report_data['net_profit'] / report_data['total_equity']
	report_data['ia'] = report_data.groupby('stock_symbol')['total_assets'].pct_change(fill_method=None)
	report_data['eps_lag1'] = report_data.groupby('stock_symbol')['basic_earnings_per_share'].shift(1)
	report_data['ue'] = report_data['basic_earnings_per_share'] - report_data['eps_lag1']
	report_data['report_type_lag1'] = report_data.groupby('stock_symbol')['report_type'].shift(1)
	report_data.to_parquet(report_data_file)
report_data

,stock_symbol,report_type,total_liabilities,net_profit,total_assets,total_equity,basic_earnings_per_share,roa,roe,ia,eps_lag1,ue,report_type_lag1
0,000001,20060331,2.342934e+11,2.331835e+08,2.395576e+11,5.264133e+09,0.12,0.000973,0.044297,NaN,NaN,NaN,None
1,000001,20060630,2.372318e+11,5.178560e+08,2.428239e+11,5.592040e+09,0.25,0.002133,0.092606,0.013635,0.12,0.13,20060331
2,000001,20060930,2.401763e+11,9.723380e+08,2.462303e+11,6.053955e+09,0.47,0.003949,0.160612,0.014028,0.25,0.22,20060630
3,000001,20061231,2.541637e+11,1.411947e+09,2.607607e+11,6.597040e+09,0.68,0.005415,0.214027,0.059011,0.47,0.21,20060930
4,000001,20070331,2.805525e+11,5.350840e+08,2.876586e+11,7.106094e+09,0.26,0.001860,0.075299,0.103152,0.68,-0.42,20061231
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255460,689009,20230331,4.161949e+09,1.734555e+07,9.204271e+09,5.042322e+09,0.24,0.001885,0.003440,-0.020163,6.35,-6.11,20221231
255461,689009,20230630,4.719132e+09,2.220691e+08,1.002299e+10,5.303860e+09,3.11,0.022156,0.041869,0.088950,0.24,2.87,20230331
255462,689009,20230930,5.449535e+09,3.783601e+08,1.081371e+10,5.364172e+09,5.29,0.034989,0.070535,0.078890,3.11,2.18,20230630
255463,689009,20231231,5.315924e+09,5.962034e+08,1.084963e+10,5.533706e+09,8.37,0.054952,0.107740,0.003322,5.29,3.08,20230930


In [3]:
# --------------------------------处理因子数据并存储为factor_data.pq--------------------------------
factor_data_file = f'{root_dir}/data/intermediate_results/factor_data.parquet'
if os.path.isfile(factor_data_file):
	factor_data_fundamentall = pq.read_table(factor_data_file).to_pandas()
else:
	factor_data_file_list = [f'{root_dir}/data/raw_data/factor_data1.csv',
							f'{root_dir}/data/raw_data/factor_data2.csv',
							f'{root_dir}/data/raw_data/factor_data3.csv',
							f'{root_dir}/data/raw_data/factor_data4.csv',
							f'{root_dir}/data/raw_data/factor_data5.csv',]
	factor_data_list = []
	for factor_data_1 in factor_data_file_list:
		factor_data_fundamentall = pd.read_csv(factor_data_1)
		factor_data_fundamentall['stock_symbol'] = factor_data_fundamentall['order_book_id'].str[:6]
		factor_data_fundamentall['trade_date'] = pd.to_datetime(factor_data_fundamentall['date']).dt.date
		factor_data_fundamentall.drop(columns=['order_book_id', 'date'], inplace=True)
		factor_data_list.append(factor_data_fundamentall)
	factor_data_fundamentall = pd.concat(factor_data_list)
	# 拼接上市日期
	factor_data_fundamentall = pd.merge(left=factor_data_fundamentall, right=stk_info[['stock_symbol', 'listed_date']], on='stock_symbol', how='left')
	factor_data_fundamentall['ln_assets'] = np.log(factor_data_fundamentall['market_cap_3'])
	factor_data_fundamentall.sort_values(by=['stock_symbol', 'trade_date'], inplace=True).reset_index(drop=True)
	factor_data_fundamentall.to_parquet(factor_data_file)
factor_data_fundamentall

,market_cap_3,book_to_market_ratio_lf,du_return_on_equity_ttm,VOL5,MTM,VOLT20,VOLT10,total_assets_mrq_0,return_on_asset_ttm,DAVOL5,...,circulation_a,non_circulation_a,total_a,free_circulation,preferred_shares,total,stock_symbol,trade_date,listed_date,ln_assets
0,1.179360e+11,0.118041,NaN,NaN,NaN,NaN,NaN,9.880178e+10,NaN,NaN,...,549193000.0,5.050807e+09,5.600000e+09,549193000.0,0.0,5.600000e+09,601688,2010-02-26,2010-02-26,25.493408
1,1.184400e+11,0.117539,NaN,NaN,NaN,NaN,NaN,9.880178e+10,NaN,NaN,...,549193000.0,5.050807e+09,5.600000e+09,549193000.0,0.0,5.600000e+09,601688,2010-03-01,2010-02-26,25.497672
2,1.164800e+11,0.119517,NaN,NaN,NaN,NaN,NaN,9.880178e+10,NaN,NaN,...,549193000.0,5.050807e+09,5.600000e+09,549193000.0,0.0,5.600000e+09,601688,2010-03-02,2010-02-26,25.480985
3,1.200080e+11,0.116003,NaN,NaN,NaN,NaN,NaN,9.880178e+10,NaN,NaN,...,549193000.0,5.050807e+09,5.600000e+09,549193000.0,0.0,5.600000e+09,601688,2010-03-03,2010-02-26,25.510824
4,1.165920e+11,0.119402,NaN,NaN,NaN,NaN,NaN,9.880178e+10,NaN,NaN,...,549193000.0,5.050807e+09,5.600000e+09,549193000.0,0.0,5.600000e+09,601688,2010-03-04,2010-02-26,25.481946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13278604,3.050105e+09,0.757138,0.56093,0.871771,0.06,0.130645,NaN,3.627149e+09,0.036145,0.582940,...,646208651.0,0.000000e+00,6.462087e+08,260052211.0,0.0,6.462087e+08,603166,2024-07-30,2014-11-27,21.838442
13278605,3.153498e+09,0.732314,0.56093,0.872548,0.01,0.133156,NaN,3.627149e+09,0.036145,0.583041,...,646208651.0,0.000000e+00,6.462087e+08,260052211.0,0.0,6.462087e+08,603166,2024-07-31,2014-11-27,21.871778
13278606,3.159960e+09,0.730816,0.56093,0.793969,0.02,0.142736,NaN,3.627149e+09,0.036145,0.530827,...,646208651.0,0.000000e+00,6.462087e+08,260052211.0,0.0,6.462087e+08,603166,2024-08-01,2014-11-27,21.873825
13278607,3.108264e+09,0.742971,0.56093,0.748091,0.07,0.146320,NaN,3.627149e+09,0.036145,0.501310,...,646208651.0,0.000000e+00,6.462087e+08,260052211.0,0.0,6.462087e+08,603166,2024-08-02,2014-11-27,21.857330


In [4]:
# --------------------------------处理指数数据并存储为index_data.pq--------------------------------
index_data_file = f'{root_dir}/data/intermediate_results/index_data.parquet'
if os.path.isfile(index_data_file):
	index_data = pq.read_table(index_data_file).to_pandas()
else:
	index_data = pd.read_excel(f'{root_dir}/data/raw_data/index_data.xlsx').iloc[2:]
	index_data['stock_symbol'] = index_data['Indexcd']
	index_data['trade_date'] = pd.to_datetime(index_data['Trddt']).dt.date
	index_data['close'] = index_data['Clsindex'].astype(float)
	index_data['rets'] = index_data['Retindex'].astype(float)
	index_data = index_data[['stock_symbol', 'trade_date', 'close', 'rets']]
	index_data.to_parquet(index_data_file)
index_data

,stock_symbol,trade_date,close,rets
2,000001,2005-12-01,1098.747,-0.000468
3,000001,2005-12-02,1094.287,-0.004059
4,000001,2005-12-05,1079.197,-0.013790
5,000001,2005-12-06,1087.794,0.007966
6,000001,2005-12-07,1099.615,0.010867
...,...,...,...,...
65971,399903,2024-12-09,3729.754,-0.000931
65972,399903,2024-12-10,3757.282,0.007381
65973,399903,2024-12-11,3751.783,-0.001464
65974,399903,2024-12-12,3787.186,0.009436


In [13]:
# --------------------------------分析师预测数据并存储为forecast_data.pq--------------------------------
forecast_data_file = f'{root_dir}/data/intermediate_results/forecast_data.parquet'
if os.path.isfile(forecast_data_file):
	forecast_data = pq.read_table(forecast_data_file).to_pandas()
else:
	forecast_data_file_list = [f'{root_dir}/data/raw_data/forecast1.xlsx',
							f'{root_dir}/data/raw_data/forecast2.xlsx',
							f'{root_dir}/data/raw_data/forecast3.xlsx',
							f'{root_dir}/data/raw_data/forecast4.xlsx']

	forecast_data_list = []
	for forecast_data_1 in forecast_data_file_list:
		forecast_data_1 = pd.read_excel(forecast_data_1, ).iloc[2:, :]
		forecast_data_1['stock_symbol'] = forecast_data_1['Stkcd']
		forecast_data_1['declare_date'] = pd.to_datetime(forecast_data_1['DeclareDate']).dt.date
		forecast_data_1['report_type'] = forecast_data_1['Fenddt'].str.replace('-','')
		forecast_data_1 = forecast_data_1[['stock_symbol', 'report_type', 'AnanmID', 'declare_date', 'Feps', 'FROA', 'FROE','TotalProfit']]
		forecast_data_list.append(forecast_data_1)
	forecast_data = pd.concat(forecast_data_list)
	# 拼接真实eps
	forecast_data = pd.merge(left=forecast_data, right=report_data[['stock_symbol', 'report_type', 'basic_earnings_per_share']], on=['stock_symbol', 'report_type'], how='left')
	# 拼接真实发报日
	forecast_data = pd.merge(left=forecast_data, right=yysj[['stock_symbol', 'report_type', '实际披露时间']], on=['stock_symbol', 'report_type'], how='left')
	forecast_data = forecast_data.sort_values(by=['stock_symbol', 'report_type', 'declare_date']).reset_index(drop=True)
	forecast_data['bias'] = forecast_data['basic_earnings_per_share'] - forecast_data['Feps']
	forecast_data['days2ann'] = forecast_data['实际披露时间'] - forecast_data['declare_date']
	forecast_data['days2ann'] = forecast_data['days2ann'].fillna(datetime.timedelta(days=999))
	forecast_data['days2ann'] = forecast_data['days2ann'].apply(lambda x: x.days)
	forecast_data.to_parquet(forecast_data_file)
forecast_data

,stock_symbol,report_type,AnanmID,declare_date,Feps,FROA,FROE,TotalProfit,basic_earnings_per_share,实际披露时间,bias,days2ann
0,000001,20021231,30264494,2002-06-28,0.200,NaN,NaN,NaN,NaN,None,NaN,999
1,000001,20021231,30248981,2002-07-18,0.301,NaN,NaN,NaN,NaN,None,NaN,999
2,000001,20031231,30266317,2003-04-04,0.290,NaN,NaN,NaN,NaN,None,NaN,999
3,000001,20031231,30265866,2003-06-04,0.280,NaN,NaN,NaN,NaN,None,NaN,999
4,000001,20041231,30245529,2004-03-29,0.200,NaN,NaN,NaN,NaN,None,NaN,999
...,...,...,...,...,...,...,...,...,...,...,...,...
2060734,900950,20131231,30243251,2012-12-27,1.771,NaN,NaN,NaN,NaN,None,NaN,999
2060735,900950,20141231,"30243299,30243292,30245083",2012-03-28,2.460,NaN,NaN,NaN,NaN,None,NaN,999
2060736,920002,20241231,30655100,2024-05-28,1.770,NaN,8.3,69000000.0,NaN,None,NaN,999
2060737,920002,20251231,30655100,2024-05-28,2.160,NaN,9.1,82000000.0,NaN,None,NaN,999


In [14]:
# --------------------------------分析师预处理测数据并存储为forecast_data_pro.pq--------------------------------
forecast_data_pro_file = f'{root_dir}/data/intermediate_results/forecast_data_pro.parquet'
if os.path.isfile(forecast_data_pro_file):
	forecast_data_pro = pq.read_table(forecast_data_pro_file).to_pandas()
else:	
	forecast_data_pro = forecast_data.groupby(['stock_symbol', 'report_type'])['bias'].mean().reset_index()
	forecast_data_pro['count'] = forecast_data.groupby(['stock_symbol', 'report_type'])['Feps'].count().values
	forecast_data_pro['fprofit'] = forecast_data.groupby(['stock_symbol', 'report_type'])['TotalProfit'].mean().values
	forecast_data_pro['eps'] = forecast_data.groupby(['stock_symbol', 'report_type'])['basic_earnings_per_share'].mean().values
	forecast_data_pro['eps'] = forecast_data_pro['eps'].replace(0,np.nan)
	forecast_data_pro['bias'] = (forecast_data_pro['bias'] / forecast_data_pro['eps']).abs()
	forecast_data_pro['bias'] = forecast_data_pro['bias'].replace(np.inf, np.nan)
	forecast_data_pro['feps_std'] = forecast_data.groupby(['stock_symbol', 'report_type'])['Feps'].std().values
	forecast_data_pro['feps_mean'] = forecast_data.groupby(['stock_symbol', 'report_type'])['Feps'].mean().values
	forecast_data_pro['feps_mean'] = forecast_data_pro['feps_mean'].replace(0,np.nan)
	forecast_data_pro['disp'] = forecast_data_pro['feps_std'] / forecast_data_pro['feps_mean']
	# forecast_data_pro['frep'] = forecast_data.groupby(['stock_symbol', 'report_type'])['AnanmID'].mean().values
	forecast_data_pro.to_parquet(forecast_data_pro_file)
forecast_data_pro

,stock_symbol,report_type,bias,count,fprofit,eps,feps_std,feps_mean,disp
0,000001,20021231,NaN,2,NaN,NaN,0.071418,0.250500,0.285101
1,000001,20031231,NaN,2,NaN,NaN,0.007071,0.285000,0.024811
2,000001,20041231,NaN,5,NaN,NaN,0.027179,0.216800,0.125365
3,000001,20051231,NaN,9,NaN,NaN,0.071673,0.191000,0.375251
4,000001,20061231,0.52402,21,NaN,0.68,0.157422,0.323667,0.486371
...,...,...,...,...,...,...,...,...,...
59731,900950,20131231,NaN,5,NaN,NaN,0.394439,1.529600,0.257871
59732,900950,20141231,NaN,1,NaN,NaN,NaN,2.460000,NaN
59733,920002,20241231,NaN,1,69000000.0,NaN,NaN,1.770000,NaN
59734,920002,20251231,NaN,1,82000000.0,NaN,NaN,2.160000,NaN
